In [3]:
#| hide

# customfunctions

> customfunctions POC Project

# Setup and Deployment Guide

## 1. Installing Snowflake CLI

[Link](https://docs.snowflake.com/en/developer-guide/snowflake-cli-v2/installation/installation)

[Cheat Sheet](https://github.com/Snowflake-Labs/sf-cheatsheets/blob/main/snowflake-cli.md)

To install the Snowflake CLI, follow the instructions provided in the official installation guide. This guide covers all the necessary steps to set up the CLI on your system.

## 2. Configuring the CLI
To use the Snowflake CLI, you need to create a connection configuration in the ~/.snowflake/config.toml file. This file allows you to specify connection details for various environments. Refer to the Snowflake CLI configuration documentation for a comprehensive guide.

[Using Snowpark in Snowflake CLI](https://docs.snowflake.com/en/developer-guide/snowflake-cli-v2/snowpark/overview)

## 3. Setting Up Your Database and Schema

You can use the Snowsight UI or the Snowflake CLI to execute the following SQL commands. While the code example uses the ACCOUNTADMIN role for simplicity, you should use a role with the appropriate permissions in your environment.

```sql
ALTER SESSION SET query_tag = '{"team":"Solutions","name":"JeremyDemlow", "version":0.1, "attributes":{"medium":"setup", "source":"customfunctions", "purpose": "setup"}}';

USE ROLE ACCOUNTADMIN;
CREATE ROLE customfunctions_USER_ROLE;

USE ROLE SYSADMIN;
GRANT CREATE DATABASE ON ACCOUNT TO ROLE customfunctions_USER_ROLE;
GRANT CREATE WAREHOUSE ON ACCOUNT TO ROLE customfunctions_USER_ROLE;
GRANT CREATE COMPUTE POOL ON ACCOUNT TO ROLE customfunctions_USER_ROLE;

USE ROLE ACCOUNTADMIN;
GRANT CREATE INTEGRATION ON ACCOUNT TO ROLE customfunctions_USER_ROLE;
GRANT MONITOR USAGE ON ACCOUNT TO ROLE customfunctions_USER_ROLE;
GRANT EXECUTE MANAGED TASK ON ACCOUNT TO ROLE customfunctions_USER_ROLE;
GRANT BIND SERVICE ENDPOINT ON ACCOUNT TO ROLE customfunctions_USER_ROLE;
GRANT IMPORTED PRIVILEGES ON DATABASE snowflake TO ROLE customfunctions_USER_ROLE;
GRANT ROLE customfunctions_USER_ROLE TO USER JDEMLOW;
GRANT ROLE customfunctions_USER_ROLE TO USER jd_service_account_admin;

USE ROLE customfunctions_USER_ROLE;

CREATE OR REPLACE WAREHOUSE customfunctions_WH
  WAREHOUSE_SIZE = XSMALL
  AUTO_SUSPEND = 120
  AUTO_RESUME = TRUE;

CREATE DATABASE customfunctions; 
CREATE SCHEMA DEV;
CREATE SCHEMA UAT;
CREATE SCHEMA PROD;
USE SCHEMA DEV;
```

## 4. Initializing and Deploying the Snowpark Project

### Creating a Boilerplate
To set up a boilerplate Snowpark project, use the following command:

```bash
snow snowpark init customfunctions
```
> Note: The boilerplate structure can be customized. In this example, adjustments were made to rename the default 'app/' directory to 'customfunctions/' in the snowflake.yml file. You can modify these settings based on your preferences.

### Building and Deploying the Project
To build and deploy the Snowpark project, execute:

```bash
snow snowpark build; snow snowpark deploy --replace; rm customfunctions.zip
```
The deployment output will list the created objects, such as procedures and functions.

## 5. Creating and Testing a Table

### Creating a Fake Orders Table
Navigate to the dev/development.ipynb notebook and run the cells to create a fake orders table in the DEV schema. This setup is essential for testing and development.

> Note: Running the entire notebook will also perform an aggregation in the DEV schema. You can defer this step if you prefer to follow the next steps.

### Testing Functions and Procedures
To test the functions and procedures, use the following commands in Snowsight or the Snowflake CLI:

```sql
-- Procedures
CALL customfunctions.DEV.HELLO_PROCEDURE('I would like a sandwich, please');
CALL customfunctions.DEV.TEST_PROCEDURE();

-- UDFs
SELECT customfunctions.DEV.HELLO_FUNCTION(PRODUCT_CATEGORY) FROM customfunctions.DEV.ORDERS;
Testing the Aggregation Procedure
To test the perform_aggregation procedure, use the following SQL commands:
```

```sql
CALL customfunctions.DEV.PERFORM_AGGREGATION(
'{
    "request_id": "AGG_001",
    "source_table": "orders",
    "target_table": "orders_aggregates",
    "group_by": ["PRODUCT_CATEGORY", "REGION"],
    "metrics": [
        {"name": "TOTAL_SALES", "function": "sum", "column": "SALES_AMOUNT"},
        {"name": "AVERAGE_ORDER_VALUE", "function": "avg", "column": "ORDER_VALUE"},
        {"name": "ORDER_COUNT", "function": "count", "column": "ORDER_ID"}
    ],
    "filters": [
        {"column": "DATE", "operator": "between", "value": ["2023-01-01", "2023-12-31"]},
        {"column": "STATUS", "operator": "in", "value": ["completed", "shipped"]}
    ],
    "version": "1.0.0"
}'
);

-- Using caller rights of the session, but using dev data and putting the results in uat
CALL customfunctions.DEV.PERFORM_AGGREGATION(
'{
    "request_id": "AGG_002",
    "source_table": "customfunctions.dev.orders",
    "target_table": "customfunctions.uat.orders_aggregates",
    "group_by": ["PRODUCT_CATEGORY", "REGION"],
    "metrics": [
        {"name": "TOTAL_SALES", "function": "sum", "column": "SALES_AMOUNT"},
        {"name": "AVERAGE_ORDER_VALUE", "function": "avg", "column": "ORDER_VALUE"},
        {"name": "ORDER_COUNT", "function": "count", "column": "ORDER_ID"}
    ],
    "filters": [
        {"column": "DATE", "operator": "between", "value": ["2023-01-01", "2023-12-31"]},
        {"column": "STATUS", "operator": "in", "value": ["completed", "shipped"]}
    ],
    "version": "1.0.0"
}'
);
```
This guide provides a streamlined approach to setting up and deploying your Snowflake environment with Snowpark. Customize as necessary to fit your project's requirements.

# NEW CHANGES & FUNCTIONS

## 1. Download SQLALCHEMY

```bash

wget https://files.pythonhosted.org/packages/py3/s/snowflake_sqlalchemy/snowflake_sqlalchemy-1.5.3-py3-none-any.whl

snow stage copy snowflake_sqlalchemy-1.5.3-py3-none-any.whl @customfunctions.DEV.CODE --overwrite
```

## 2. Deploy Snowpark Project

```bash

snow snowpark build; snow snowpark deploy --replace; rm customfunctions.zip; rm requirements.snowflake.txt

customfunctions % snow snowpark build; snow snowpark deploy --replace; rm customfunctions.zip; rm requirements.snowflake.txt
Build done. Artifact path: /Users/jdemlow/github/customfunctions/customfunctions.zip
+----------------------------------------------------------------------------------------+
| object                                                | type      | status             |
|-------------------------------------------------------+-----------+--------------------|
| customfunctions.DEV.hello_procedure(name string)                  | procedure | definition updated |
| customfunctions.DEV.perform_aggregation(aggrequest string)        | procedure | definition updated |
| customfunctions.DEV.test_procedure()                              | procedure | definition updated |
| customfunctions.DEV.perform_aggregation_sqlach(aggrequest string) | procedure | created            |
| customfunctions.DEV.hello_function(name string)                   | function  | definition updated |
+----------------------------------------------------------------------------------------+
```

## Call Command With SqlAlchemy

```sql

CALL customfunctions.DEV.PERFORM_AGGREGATION_SQLACH(
'{
    "request_id": "AGG_001",
    "source_table": "orders",
    "target_table": "customfunctions.prod.orders_aggregates",
    "group_by": ["PRODUCT_CATEGORY", "REGION"],
    "metrics": [
        {"name": "TOTAL_SALES", "function": "sum", "column": "SALES_AMOUNT"},
        {"name": "AVERAGE_ORDER_VALUE", "function": "avg", "column": "ORDER_VALUE"},
        {"name": "ORDER_COUNT", "function": "count", "column": "ORDER_ID"}
    ],
    "filters": [
        {"column": "DATE", "operator": "between", "value": ["2023-01-01", "2023-12-31"]},
        {"column": "STATUS", "operator": "in", "value": ["completed", "shipped"]}
    ],
    "version": "1.0.0"
}'
);

```

```bash
# Results
Aggregation completed. Results saved to customfunctions.prod.orders_aggregates

```

> If you want to use snowflake notebooks please let me know and I will show you that method here, but this should be good to go now.